In [2]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

In [3]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [4]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [5]:
semana = 33

In [6]:
query_semana = {'query': 'select numero_registro, nom_ca  from dataset_318806 where extract(week from fecha_registro) = {sem}'.format(sem=semana)}
project = 'https://app.amigocloud.com/api/v1/projects/33075/sql'
select_semana = amigocloud.get(project, query_semana)
data_semana = select_semana['data']

In [7]:
data_semana

[{'numero_registro': 69, 'nom_ca': '2893 / DAVALOS SILVA MARIO '},
 {'numero_registro': 67, 'nom_ca': '8226 / MEDRANO NUNEZ JUAN'},
 {'numero_registro': 70, 'nom_ca': '2042 / CKOLO ESCALERA ABIGAIL'},
 {'numero_registro': 68, 'nom_ca': '153 / ALVAREZ DURAN FILIBERTO '}]

In [9]:
for i in data_semana:
    id_insp = i['numero_registro']
    # DATOS DE LAS INPECCION
    query = {'query': 'select * from dataset_318806 where numero_registro={id}'.format(id=id_insp)}
    project = 'https://app.amigocloud.com/api/v1/projects/33075/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]
    data['fecha_registro'] = datetime.strptime(data['fecha_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    data['nom_ca'] = data['nom_ca'].title()
    canhero = collections.namedtuple("canhero", data.keys())(*data.values())
    
    query = {'query': 'select * from dataset_318811 where prop_to_visit = \'{amigo_id}\''.format(amigo_id=canhero.amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/33075/sql'
    select = amigocloud.get(project, query)
    data = select['data']
    
    trabajadores = []
    for i in data:
        i['fecha_nacimiento'] = datetime.strptime(i['fecha_nacimiento'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
        if i['otro_antecedente'] is None:
            i['otro_antecedente'] = ''
        if i['enfermedad_actual'] is None:
            i['enfermedad_actual'] = ''
        if i['otra_exploracion'] is None:
            i['otra_exploracion'] = ''
        if i['observaciones_diagnostico'] is None:
            i['observaciones_diagnostico'] = ''
        if i['tratamiento'] is None:
            i['tratamiento'] = ''
        if i['observaciones_tratamiento'] is None:
            i['observaciones_tratamiento'] = ''
        trab = collections.namedtuple("trabajador", i.keys())(*i.values())
        trabajadores.append(trab)
    
    for trabajador in trabajadores:
        doc = DocxTemplate(ruta + '/templates/tpl_rpt_examen_seguimiento-bpl.docx')
        context = {'canhero':canhero, 'trabajador':trabajador}
        nom_file = canhero.nom_ca.replace('/', '-').strip() + ' - ' + str(trabajador.number_registro) + ' - ' + trabajador.nom_trabajador
        doc.render(context)
        doc.save(nom_file + '.docx')

In [84]:
id_insp = 1
# DATOS DE LAS INPECCION
query = {'query': 'select * from dataset_318806 where numero_registro={id}'.format(id=id_insp)}
project = 'https://app.amigocloud.com/api/v1/projects/33075/sql'
select = amigocloud.get(project, query)
data = select['data'][0]
data['fecha_registro'] = datetime.strptime(data['fecha_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
data['nom_ca'] = data['nom_ca'].title()
canhero = collections.namedtuple("canhero", data.keys())(*data.values())

In [86]:
query = {'query': 'select * from dataset_318811 where prop_to_visit = \'{amigo_id}\''.format(amigo_id=canhero.amigo_id)}
project = 'https://app.amigocloud.com/api/v1/projects/33075/sql'
select = amigocloud.get(project, query)
data = select['data']

In [87]:
trabajadores = []
for i in data:
    i['fecha_nacimiento'] = datetime.strptime(i['fecha_nacimiento'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    if i['otro_antecedente'] is None:
        i['otro_antecedente'] = ''
    if i['enfermedad_actual'] is None:
        i['enfermedad_actual'] = ''
    if i['otra_exploracion'] is None:
        i['otra_exploracion'] = ''
    if i['observaciones_diagnostico'] is None:
        i['observaciones_diagnostico'] = ''
    if i['tratamiento'] is None:
        i['tratamiento'] = ''
    if i['observaciones_tratamiento'] is None:
        i['observaciones_tratamiento'] = ''
    trab = collections.namedtuple("trabajador", i.keys())(*i.values())
    trabajadores.append(trab)

In [114]:
for trabajador in trabajadores:
    doc = DocxTemplate(ruta + '/templates/tpl_rpt_examen_seguimiento.docx')
    context = {'canhero':canhero, 'trabajador':trabajador}
    nom_file = canhero.nom_ca.replace('/', '-').strip() + ' - ' + str(trabajador.number_registro) + ' - ' + trabajador.nom_trabajador
    doc.render(context)
    doc.save(nom_file + '.docx')